## קוד מבוא

### ספריות

In [15]:
import os
import sys
import pandas as pd
import geopandas as gpd
import fiona
from openpyxl import load_workbook

In [16]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### העלת משתנים להרצת הקוד

In [17]:
path = os.getcwd()

software_root_folder = os.path.dirname(path)

software_folder_location = r'{}\current'.format(software_root_folder)

df_inputs_outputs = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(software_folder_location))

forecast_version_folder_location=df_inputs_outputs['location'][1]

sys.path.append(software_root_folder)

## פונקציות

In [18]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [19]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [20]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [21]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [22]:
def unique_id_in_one_taz(df_to_geoode,unique_field,gpd_for_geocode,taz):
    
    code_to_find=list(df_to_geoode[unique_field].unique())

    gpd_for_geocode_to_sum_by_taz=gpd_for_geocode[[unique_field,'geometry']].loc[gpd_for_geocode[unique_field].isin(code_to_find)]

    gpd_for_geocode_to_sum_by_taz=gpd.sjoin(taz[['Taz_num','geometry']],gpd_for_geocode_to_sum_by_taz)

    gpd_for_geocode_in_one_taz=list(gpd_for_geocode_to_sum_by_taz.groupby(unique_field)[['Taz_num']].nunique().query('Taz_num==1').reset_index()[unique_field])

    return gpd_for_geocode_in_one_taz

In [23]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df

## 

### העלת מידע

In [24]:
stat=up_load_shp(r'{}\background_files\stat_11_cbs_2020_with_jtmt_fix.shp'.format(software_folder_location))

In [25]:
stat_in_taz=stat.loc[stat['type_stat']=='in_taz']

In [26]:
stat=stat.loc[stat['type_stat']!='in_taz']

In [27]:
df_inputs_outputs.loc[1]['location']

'True'

In [ ]:
taz=up_load_shp(r'{}\background_files\TAZ_V4_230518_Published.shp'.format(software_folder_location))

In [ ]:
if df_inputs_outputs.loc[2]['location'] == True:
    taz=up_load_shp(df_inputs_outputs.loc[3]['location'])
else:
    #load excel file
    workbook = load_workbook(filename=r'{}\inputs_outputs.xlsx'.format(software_folder_location))

    #open workbook
    sheet = workbook.active

    #modify the desired cell
    sheet["B5"] = ''

    #save the file
    workbook.save(filename=r'{}\inputs_outputs.xlsx'.format(software_folder_location))

In [ ]:
path=r'{}\background_files\BNTL_2022.gdb'.format(software_folder_location)
bld=up_load_gdb(path,'BLDG_Clip')

In [ ]:
path=r'{}\background_files\BNTL_2022.gdb'.format(software_folder_location)
bld_poi=up_load_gdb(path,'POI_BLDG_Clip')

In [ ]:
taz_not_relevant=up_load_df(r'{}\background_files'.format(software_folder_location),'taz_not_relevant_for_pop_2020')

In [ ]:
taz_not_relevant=list(taz_not_relevant.Taz_num)

### שכבת בניינים

In [ ]:
bld=bld.merge(bld_poi[[ 'FCODE', 'USG_GROUP', 'USG_CODE','USG_SP_NM_LTN','BLDG_ID']],how='left',left_on='UNIQ_ID',right_on='BLDG_ID')

In [ ]:
bld=bld.loc[(bld['FCODE_y'].isna())|(bld['USG_CODE']==7600)] #זה קוד לבניינים מעורב שימושים אני מניח שעדיף לשים בניינים מיותרים מאשר הפוך

In [ ]:
bld['bld_area']=bld.area

In [ ]:
bld_point=make_point(bld).fillna(0)

In [ ]:
bld_point.loc[bld_point['HEIGHT']<=0,'HEIGHT']=bld_point['HI_PNT_Z']-bld_point['HT_LAND']

In [ ]:
ceiling_height=3

In [ ]:
bld_point['bld_m3']=(bld_point['HEIGHT']/ceiling_height).astype(int)*bld_point['bld_area'].astype(int)

In [ ]:
bld_point.loc[(bld_point['HEIGHT']-ceiling_height)<=0,'bld_m3']=bld_point['bld_area']

### הורדת מבנים לא רלוונטים

In [ ]:
bld_point=gpd.sjoin(bld_point,taz.loc[(~taz['Taz_num'].isin(taz_not_relevant))&(taz['Palestinia']==0)])

In [ ]:
col=['bld_m3','centroid','Taz_num']

In [ ]:
bld_point=bld_point[col]

In [ ]:
bld_point=gpd.sjoin(bld_point,stat[['STAT','geometry']])

In [ ]:
bld_point['bld_m3']=bld_point['bld_m3'].astype(int)

### יצירת מאה אחוז לכל א"ס

In [ ]:
sum_bld_m3=pd.pivot_table(bld_point.drop(columns='centroid'),index=['Taz_num','STAT'],aggfunc=sum)[['bld_m3']].reset_index()

In [ ]:
sum_bld_m3_by_stat=sum_bld_m3.pivot_table(index='STAT',aggfunc=sum)[['bld_m3']].rename(columns={'bld_m3':'bld_m3_stat'}).reset_index()

In [ ]:
sum_bld_m3=sum_bld_m3.merge(sum_bld_m3_by_stat,on='STAT')

In [ ]:
sum_bld_m3['precent_of_stat_data']=sum_bld_m3['bld_m3']/sum_bld_m3['bld_m3_stat']

In [ ]:
sum_bld_m3=sum_bld_m3.loc[(sum_bld_m3['precent_of_stat_data']>0.01)|(sum_bld_m3['Taz_num']==2001)]

In [ ]:
sum_bld_m3_by_stat=sum_bld_m3.pivot_table(index='STAT',aggfunc=sum)[['bld_m3']].rename(columns={'bld_m3':'bld_m3_stat_new'}).reset_index()

In [ ]:
sum_bld_m3=sum_bld_m3.merge(sum_bld_m3_by_stat,on='STAT')

In [ ]:
sum_bld_m3['precent_of_stat_data']=sum_bld_m3['bld_m3']/sum_bld_m3['bld_m3_stat_new']

### א"ס ללא פוליגון



In [ ]:
stat_in_taz=make_point(stat_in_taz).sjoin(taz[['Taz_num','geometry']])[['STAT','Taz_num']]

In [ ]:
stat_in_taz['precent_of_stat_data']=1

### איחוד לטבלה אחת



In [ ]:
col=['Taz_num', 'STAT',  'precent_of_stat_data']

In [ ]:
precent_of_stat_data=pd.concat([sum_bld_m3[col], stat_in_taz])

In [ ]:
# precent_of_stat_data.to_excel(r'\\FILE-SRV\Jtmt\projections_team\כללי\פעולות\יצירת_מצב_קיים_למס\output\precent_of_stat_data.xlsx',index=False)
precent_of_stat_data.to_excel(r'{}\Intermediates\precent_of_stat_data.xlsx'.format(software_folder_location),index=False)

###  יצירת מידע ברמת אזור תנועה

In [ ]:
# stat=up_load_df(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2020\מרכיבי_תחזית','stat_cbs_jtmt_2020')
stat=up_load_df(r'{}\Intermediates'.format(software_folder_location),'stat_cbs_jtmt_2020')

In [ ]:
col=['pop_0',
 'pop_5',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up']

In [ ]:
stat[col]=stat[col].multiply(stat['pop'], axis="index")

In [ ]:
stat['pop_hardi']=stat['pop']*(stat['pre_hardi']/100)

In [ ]:
taz_stat_conver=precent_of_stat_data

In [ ]:
col=['STAT',
 'aprt_20',
 'pop_0',
 'pop_5',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up',
 'pop',
 'pop_hardi']

בשכבת האזורי תנועה יהיו רק אזורי תנועה שמפצלים את המידע של למס

In [ ]:
taz=pd.merge(taz_stat_conver,stat[col],on='STAT',how='left')

In [ ]:
col=['aprt_20',
 'pop_0',
 'pop_5',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up',
 'pop',
 'pop_hardi']

In [ ]:
taz[col]=taz[col].multiply(taz['precent_of_stat_data'], axis="index")

In [ ]:
taz=pd.pivot_table(taz,index='Taz_num',aggfunc=sum)

In [ ]:
taz['pre_hardi']=taz['pop_hardi']/taz['pop']

In [ ]:
taz['hh_size']=taz['pop']/taz['aprt_20']

In [ ]:
# save_taz_path=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2020\מרכיבי_תחזית'
save_taz_path=r'{}\Intermediates'.format(software_folder_location)

In [ ]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [ ]:
taz.to_excel(r'{}\taz_with_pop_info.xlsx'.format(save_taz_path,file_date))